
<div style="font-size: 25px">
    
**Исследование надёжности заёмщиков**
</div >


<div> Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.</div>

<div>Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку. </div>

<div>Откройте файл с данными и изучите общую информацию.</div>

In [2]:
import pandas as pd
data = pd.read_csv('data.csv')  
data.info()
data.head(20)

FileNotFoundError: [Errno 2] No such file or directory: 'data.csv'

<div>Прочитаем файл data.csv и сохраним его в переменной data.
Количество значений в столбцах различается. Это говорит о том, что в данных есть <нулевые> значения. Пропуски могут возникнуть в результате ошибки при выгрузке данных или при заполнении данных в таблицу. </div>

<div>1. Предобработка данных</div>

<div>Обработка пропусков</div>


In [3]:
data.isnull().sum()   # проверяем данные на пропуски
data['days_employed'].head(20)

NameError: name 'data' is not defined

In [3]:
for i in range(len(data['days_employed'])):  # замена отрицательных значений на положительные
    if data.loc[i,'days_employed'] < 0:
        data.loc[i,'days_employed'] = data.loc[i,'days_employed'] * -1
data[data['days_employed'] < 0]['days_employed'].count()   # проверка отсутствия отрицатльных значений

0

In [4]:
days_employed_max = (65 - 18) * 365          # вычисление максимально возможного рабочего стажа
for i in range(len(data['days_employed'])):  # поиск и перевод значений больше максимального рабочего стажа из дней в годы
    if data.loc[i,'days_employed'] > days_employed_max:
        data.loc[i,'days_employed'] = data.loc[i,'days_employed'] / 24
data[data['days_employed'] > days_employed_max]['days_employed'].count() # проверка отсутствия значений больше максимального 
                                                                         # рабочего стажа

0

In [5]:
def ages_group(ages): # категоризация по возрастному признаку
    if ages < 21:
        return 'young_period'
    if 21 <= ages < 35:
        return 'mature_period1'
    if 35 <= ages < 60:
        return 'mature_period2'
    if ages >= 60:
        return 'elderly_period'
    
    
data['ages_group']=data['dob_years'].apply(ages_group) # добавление в таблицу столбца с категориями по возрастному признаку 

In [6]:

young_period_mean = data.loc[data['dob_years'] < 21]['days_employed'].mean() # находим среднее значение по категории


data_mature_period1 = data.loc[(data['dob_years'] >= 21) & (data['dob_years'] < 35)]  # находим среднее значение по категории 
mature_period1_mean = data_mature_period1['days_employed'].mean()



data_mature_period2 = data.loc[(data['dob_years'] >= 35) & (data['dob_years'] < 60)] # находим среднее значение по категории
mature_period2_mean = data_mature_period2['days_employed'].mean()



elderly_period_mean = data[data['dob_years'] >= 60]['days_employed'].mean() # находим среднее значение по категории
elderly_period_mean

12728.438251169182

In [7]:
data['days_employed'] = data['days_employed'].fillna(0) # замена пропусков на '0'

for i in range(len(data['days_employed'])):    # замена нулевых значений усреднеными, в зависимости от категории
    if data.loc[i,'days_employed'] == 0.000000 and data.loc[i,'ages_group'] == 'young_period':   
        data.loc[i,'days_employed'] = young_period_mean
    if data.loc[i,'days_employed'] == 0.000000 and data.loc[i,'ages_group'] == 'mature_period1':
        data.loc[i,'days_employed'] = mature_period1_mean
    if data.loc[i,'days_employed'] == 0.000000 and data.loc[i,'ages_group'] == 'mature_period2':
        data.loc[i,'days_employed'] = mature_period2_mean
    if data.loc[i,'days_employed'] == 0.000000 and data.loc[i,'ages_group'] == 'elderly_period':
        data.loc[i,'days_employed'] = elderly_period_mean
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [8]:
def income_type_group(income_type): # категоризация по типу занятости
    if income_type == 'предприниматель' or income_type == 'компаньон':
        return 'to_myself'
    if income_type == 'сотрудник' or income_type == 'госслужащий':
        return 'worker'
    else: 
        return 'No_work'

data['income_type_group']=data['income_type'].apply(income_type_group)  # добавление в таблицу столбца с категориями 
                                                                        #по типу занятости

In [9]:
data['total_income'] = data['total_income'].fillna(0)                     # замена пропусков на '0'


to_myself_mean_income = data.loc[(data['income_type_group'] == 'to_myself')]['total_income'].mean() # находим среднее значение по категории
worker_mean_income = data.loc[(data['income_type_group'] == 'worker')]['total_income'].mean()  # находим среднее значение по категории
No_work_mean_income = data.loc[(data['income_type_group'] == 'No_work')]['total_income'].mean()  # находим среднее значение по категории


In [10]:
for i in range(len(data['total_income'])): # замена нулевых значений усреднеными, в зависимости от категории
    if data.loc[i,'total_income'] == 0.000000 and data.loc[i,'income_type_group'] == 'to_myself':  
        data.loc[i,'total_income'] = to_myself_mean_income
    if data.loc[i,'total_income'] == 0.000000 and data.loc[i,'income_type_group'] == 'worker': 
        data.loc[i,'total_income'] = worker_mean_income
    if data.loc[i,'total_income'] == 0.000000 and data.loc[i,'income_type_group'] == 'No_work': 
        data.loc[i,'total_income'] = No_work_mean_income
        
data.isnull().sum()

children             0
days_employed        0
dob_years            0
education            0
education_id         0
family_status        0
family_status_id     0
gender               0
income_type          0
debt                 0
total_income         0
purpose              0
ages_group           0
income_type_group    0
dtype: int64

In [11]:

data[data['dob_years'] == 0]['dob_years'].count()

name_columns = data.columns

for column in name_columns:  # поиск количесва нулевых значений каждого столбца
    null = data[data[column] == 0][column].count()
    print('Количество нулевых значений в столбце',column,null)

Количество нулевых значений в столбце children 14149
Количество нулевых значений в столбце days_employed 0
Количество нулевых значений в столбце dob_years 101
Количество нулевых значений в столбце education 0
Количество нулевых значений в столбце education_id 5260
Количество нулевых значений в столбце family_status 0
Количество нулевых значений в столбце family_status_id 12380
Количество нулевых значений в столбце gender 0
Количество нулевых значений в столбце income_type 0
Количество нулевых значений в столбце debt 19784
Количество нулевых значений в столбце total_income 0
Количество нулевых значений в столбце purpose 0
Количество нулевых значений в столбце ages_group 0
Количество нулевых значений в столбце income_type_group 0


In [12]:
data.loc[data['dob_years'] == 0, 'dob_years']=[pd.np.nan] #замена 0 на  пропуски 'nan'
data.dropna(subset = ['dob_years'],inplace = True) # удаление пропусков
data.isnull().sum() # проверяем данные на пропуски
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,ages_group,income_type_group
0,1,8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,mature_period2,worker
1,1,4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,mature_period2,worker
2,0,5623.422610,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,mature_period1,worker
3,3,4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,mature_period1,worker
4,0,14177.753002,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,mature_period2,No_work
5,0,926.185831,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,mature_period1,to_myself
6,0,2879.202052,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,mature_period2,to_myself
7,0,152.779569,50.0,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,mature_period2,worker
8,2,6929.865299,35.0,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,mature_period2,worker
9,0,2188.756445,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,mature_period2,worker


<div>Проверим данные на наличие пропусков вызовом набора методов для суммирования пропущенных значений isnull().sum().
Общий трудовой стаж и ежемесячный доход пропущен в 2174 случаях. Это 10% от всех значений датафрейма data, удаление такой доли строк повлияет на результаты.

Внимательнее изучим значения общего трудового стажа.
- Первым делом замечаем отрицательные значения, что выглядит странным. Чтобы избавиться от этого артефакта, сравним все значения столбца 'days_employed' с '0' и все, что '< 0'. умножим на '-1' и проверим результат.
- Затем обращаем внимание на то, что в большом количестве строк черезчур большие значения. Предпологая, что общий трудовой стаж заполнен в днях, эти значения при пересчете на года (поделив на 365) дают 800-1100 лет опыта, что неприемлемо. Значит делаем предположение, что эти данные заполнены в часах, тогда делением на 24 приведем эти значения к приемлемому виду. Чтобы расчитать максимальный трудовой стаж вычтем от возраста (65 лет) выхода на пенсию мужчины  минимальный возраст (18 лет) трудовой деятельности  и переведем в дни. Чтобы определить какие значения черезмерно большие, сравним все значения столбца 'days_employed' c масимальным значением трудового стажа и те, что больше разделим на 24. Проверим результат.
- Приведя значения столбца 'days_employed' в порядок, приступим к замене пропусков. Для корректной замены разобьем данные по следующим возрастным категориям [young_period: до 21 года],[mature_period1: 21 — 35 лет],[mature_period2: 35 — 60 лет],[elderly_period: 60 лет и старше], и для каждой категории найдем среднее значение трудового стажа. После чего методом 'fillna' заменим пропуски нулевыми значениями и затем заменив эти нулевые значения усредненными, в зависимости от категори по возврасту.

Теперь избавимся от пропусков в значениях ежемесячного дохода.
- Для начала разобьем потенциальных заемщиков по типу занятости. Посмотрим уникальные значения столбца 'income_type' методом 'unique'. Разбиваем на следующие категории: [No_work: 'пенсионер', 'студент', 'в декрете', 'безработный'],[to_myself: 'предприниматель', 'компаньон'],[worker:'сотрудник', 'госслужащий']. И по аналогии с пропусками в значениях общего трудового стажа заменяем пропуски на усредненые значения, в зависимости от категории.

Помимо явных пропусков проверим таблицу на некорректные нулевые значения. Проверив все столбцы на нулевые значения видим 101 ноль в столбце 'dob_years', что является некорректным. Так как количество нулевых значений не большое, можем удалить данные строки из таблицы предварительно заменив их на пропуски 'nan'.

В процессе предобработки данных мы встретили пропуски и артефакты, такие как отрицательные значения трудового стажа, и черезмерно большие значения стажа. При замене пропусков применили категоризацию по возрасту, так как показалась наибольшая зависимость трудового стажа от возраста заемщика, но так же можно было произвести категоризацию и по типу занятости.
При замене пропусков в значениях ежемесячного дохода произвели категоризацию по типу занятости, т.к доходы заемщиков наиболее зависят от их работы или ее отсутствия.Из странностей в данных так же нашли нулевые значения в значениях возраста и учитывая, что таких значений не много решили удалить эти строки. Если бы таких значений было много, мы бы заменили их на примерный возраст, опираясь на данные трудового стажа.
Описаные недочеты в данных могли возникнуть как в результате человеческого фактора так и при загрузке/выгрузке данных. 
По окончании удаления пропусков и исправления артефактов получаем таблицу, без пропусков, без отрицательных значений трудового стажа и заемщиков с ворастом 0 лет, готовую к дальнейшей предобработке и проведению исследования.</div>

<div>Замена типа данных</div>

In [2]:
data['days_employed']=data['days_employed'].astype('int') #замена типа данных на целочисленный
data['total_income']=data['total_income'].astype('int')   
data.head()


ValueError: Cannot convert non-finite values (NA or inf) to integer

<div>Значения общего трудового стажа представлены вещественные числами, что не является корректным, заменим их на целочисленные значения методом astype('int'). Так же значения 'total_income' не нуждаются в такой высокой точности, поэтому их тоже заменим целочисленными значениями.</div>


<div>Обработка дубликатов</div>

In [15]:
data.duplicated().sum() # нахождение количества дубликатов
data['education'].unique()  # проверка уникальных значений столбца 'образование'
data['education']=data['education'].str.lower() # приведение всех значений к нижнему регистру
data = data.drop_duplicates().reset_index(drop = True)  # удаление дубликатов, с восстановлением индексов
data.duplicated().sum() # проверка, что дубликаты отсутствуют



0

In [16]:
data.duplicated().sum()

0

<div>Необходимо установить наличие дубликатов. Если найдутся, удаляем, и проверяем, все ли удалились.
Для выявления неявных дубликатов проверим уникальные значения в столбцах, выявленных специальным методом unique().
В столбце 'education' видим значения в разных регистрах, приведем их всех к нижнему регистру вызовом метода str.lower().
Проверяем наличие дубликатов вызовом метода duplicated().sum().
Обнаружив наличие дупликатов удаляем их методом drop_duplicates() с восстановлением индексов reset_index(drop = True).
Убеждаемся, что удалили все дубликаты.
Дубликаты могли появиться вследствие сбоя в записи данных или из-за человеческого фактора.</div>

<div>Лемматизация</div>

In [17]:
from pymystem3 import Mystem 
m = Mystem()
for purpose in data['purpose'].unique(): # перебор уникальных значений списка целей кредита
    lemmas = m.lemmatize(purpose)  # выделение лемм уникальных значений
    print(''.join(lemmas))

покупка жилье

приобретение автомобиль

дополнительный образование

сыграть свадьба

операция с жилье

образование

на проведение свадьба

покупка жилье для семья

покупка недвижимость

покупка коммерческий недвижимость

покупка жилой недвижимость

строительство собственный недвижимость

недвижимость

строительство недвижимость

на покупка подержать автомобиль

на покупка свой автомобиль

операция с коммерческий недвижимость

строительство жилой недвижимость

жилье

операция со свой недвижимость

автомобиль

заниматься образование

сделка с подержанный автомобиль

получение образование

автомобиль

свадьба

получение дополнительный образование

покупка свой жилье

операция с недвижимость

получение высокий образование

свой автомобиль

сделка с автомобиль

профильный образование

высокий образование

покупка жилье для сдача

на покупка автомобиль

ремонт жилье

заниматься высокий образование



 <div>Выделяем леммы в значениях столбца с целями получения кредита методом lemmatize() для дальнейшей категоризации.</div>

<div>Категоризация данных</div>

In [18]:
data['children'].value_counts()
data['children']=data['children'].replace(-1,1).replace(20,2) # замена некорректных данных

def children_group(children):           # категоризация по количеству детей
    if children == 0:
        return 'Нет детей'
    if children == 1 or children == 2:
        return 'Обычное кол-во детей(1-2 ребенка)'
    if children >= 3: 
        return 'Большое кол-во детей(больше 2х)'

In [19]:
data = data.sort_values(by = 'total_income',ascending = True)  # сортируем таблицу по возрастанию дохода
data['total_income_group'] = pd.qcut(range(len(data['total_income'])), 7, labels=["group_1", "group_2",
                                        "group_3","group_4","group_5", "group_6", "group_7"])  # категоризация по ежемесячному доходу
data.sort_index(inplace=True) # сортируем таблицу по индексу

In [20]:
from pymystem3 import Mystem            # категоризация по цели кредитования
m = Mystem()
def purpose_group(purpose):
    lemmas = m.lemmatize(purpose)
    if 'автомобиль' in lemmas:
        return 'Кредит на автомобиль'
    if 'образование' in lemmas:
        return 'Кредит на образование'
    if 'недвижимость' in lemmas or 'жилье' in lemmas:
        return 'Кредит на недвижимость'
    if 'свадьба' in lemmas:
        return 'Кредит на свадьбу' 
    

In [21]:
    
# Добавляем в 'data'столбцы с сгруппированными по категориям данными.
data['children_group']=data['children'].apply(children_group)
data['purpose_group']=data['purpose'].apply(purpose_group)

data.head(15)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,ages_group,income_type_group,total_income_group,children_group,purpose_group
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,mature_period2,worker,group_7,Обычное кол-во детей(1-2 ребенка),Кредит на недвижимость
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,mature_period2,worker,group_2,Обычное кол-во детей(1-2 ребенка),Кредит на автомобиль
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,mature_period1,worker,group_4,Нет детей,Кредит на недвижимость
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,mature_period1,worker,group_7,Большое кол-во детей(больше 2х),Кредит на образование
4,0,14177,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,mature_period2,No_work,group_5,Нет детей,Кредит на свадьбу
5,0,926,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,mature_period1,to_myself,group_7,Нет детей,Кредит на недвижимость
6,0,2879,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,mature_period2,to_myself,group_6,Нет детей,Кредит на недвижимость
7,0,152,50.0,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,mature_period2,worker,group_3,Нет детей,Кредит на образование
8,2,6929,35.0,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,mature_period2,worker,group_2,Обычное кол-во детей(1-2 ребенка),Кредит на свадьбу
9,0,2188,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,mature_period2,worker,group_4,Нет детей,Кредит на недвижимость


<div>Разобьем на категории людей по количеству детей. Для этого уточним сколько раз упоминается количество детей в семье методом value_counts(), видим странные значения (20) и (-1).</div>

<div>Предпологаем, что эти значения появилсь в следствии ошибки при наборе и их реальные значения соответственно (2) и (1). Исправим эти значения методом replace.</div>

<div>По количеству детей разбиваем на категории: 'Нет детей', 'Обычное кол-во детей', 'Большое кол-во детей'.</div>

<div>По ежемесячному доходу разбиваем на 7 групп, для уменьшения разброса данных по группам, методом 'qcut', чем выше номер группы, тем больше доход.</div>

<div>Изучив леммы целей получения кредита, можем разбить эти цели на категории: 'Кредит на автомобиль', 'Кредит на образование', 'Кредит на недвижимость' и 'Кредит на свадьбу'</div>

<div>Добавляем в датафрейм 'data' столбцы с сгруппированными по категориям данными методом 'apply'.</div>



<div>Ответьте на вопросы</div>

<div>Есть ли зависимость между наличием детей и возвратом кредита в срок?</div>

In [23]:
data_children=data[['children_group','debt']]
data_children_groupby = data_children.groupby('children_group').sum()
data_children_groupby['quantity'] = data_children['children_group'].value_counts()
try:
    data_children_groupby['perсent'] = (data_children_groupby['debt'] / data_children_groupby['quantity']).apply('{:.1%}'.format)
    print(data_children_groupby)
except:
    print('Проверьте правильность данных')


                                   debt  quantity perсent
children_group                                           
Большое кол-во детей(больше 2х)      31       378    8.2%
Нет детей                          1058     14022    7.5%
Обычное кол-во детей(1-2 ребенка)   644      6953    9.3%


<div>Составив сводную таблицу с суммой задолжностей и количеством детей у потенциальных заемщиков.
Наблюдаем, что люди с 1-2 детьми чаще имели задолжности перед банками, а люди без детей реже имели задолжности.
Люди не имеющие детей чаще всего обращаются за кредитами, что можно объяснить тем, что люди без детей меньше предрасположены к незапланированным тратам и могут быть более уверенными в своей платежеспособности, поэтому и вероятность задолжностей самая низкая. 
Задолжности группы с обычным количеством детей больше на 1.1 % чем у тех у кого более 2х детей, возможно из-за неопытности молодых родителей в планировании своих расходов, так и из-за того, что люди с большим количеством детей берут кредит только когда уверены, что смогут его вовремя погасить,что подтверждается и самым низким количеством взятия кретитов. 
Так как разница между группами равна примерно 1%, учитывая объемы таблицы, можно сделать вывод, что зависимость между наличием детей и возвратом кредита в срок значительна.</div>


<div>Есть ли зависимость между семейным положением и возвратом кредита в срок?</div>

In [24]:
data_family=data[['family_status','debt']]
data_family_groupby = data_family.groupby('family_status').sum()
data_family_groupby['quantity'] = data_family['family_status'].value_counts()

try:
    data_family_groupby['perсent'] = (data_family_groupby['debt'] / data_family_groupby['quantity']).apply('{:.1%}'.format)
    print(data_family_groupby)
except:
    print('Проверьте правильность данных')

                       debt  quantity perсent
family_status                                
Не женат / не замужем   273      2794    9.8%
в разводе                85      1185    7.2%
вдовец / вдова           62       954    6.5%
гражданский брак        386      4130    9.3%
женат / замужем         927     12290    7.5%


<div>По сводной таблице семейного положения и наличия задолжностей наблюдаем, что зависимость довольно велика, разница более 3%.
Больше остальных берут кредиты люди находящиеся в браке, вероятнее всего из - за более стабильного бюджета,т.к в случае изменения доходов одного из семьи, сохраняется доход у второго, да и жизнь в браке предпологает постоянное улучшение качества жизни, с сопутсвующими затратами.
Наивысшим процентом по наличию задолжностей владеют неженатые люди и люди живущие в гражданском браке. В первом случае, возможно, люди не находящиеся в браке берут кредит и попадая в финансовую нестабильность не имют финансовой подержки, из-за чего появляются задолжности. Часто не женатые люди - это молодые люди, не умеющие планировать свой бюджет, что тоже приводит к задолжностям. Люди живущие в гражданском браке чаще расходятся, чем люди женатые, тем самым по кредитам взятым 'на двоих' часто возникают задолжности.</div>


<div> Есть ли зависимость между уровнем дохода и возвратом кредита в срок?</div>

In [25]:
data_total_income_groupby = data[['total_income_group','debt']]
data_total_income_groupby=data_total_income_groupby.groupby('total_income_group').sum()
data_total_income_groupby['quantity'] = data['total_income_group'].value_counts()
try:
    data_total_income_groupby['persent'] = (data_total_income_groupby['debt'] / data_total_income_groupby['quantity']).apply('{:.1%}'.format)
    print(data_total_income_groupby)
except:
    print('Проверьте правильность данных')



                    debt  quantity persent
total_income_group                        
group_1              233      3051    7.6%
group_2              261      3050    8.6%
group_3              263      3050    8.6%
group_4              266      3051    8.7%
group_5              254      3050    8.3%
group_6              246      3050    8.1%
group_7              210      3051    6.9%


<div>По сводной таблице уровня дохода(чем выше номер группы тем выше доход) и наличия задолжностей наблюдаем значительную зависимость между уровнем дохода и возвратом кредита в срок. Люди со средними доходами имеют разницу всего в 0.5%, а люди с самыми высоким и самыми низкими доходами отличаются от них на 1%, что достаточно много.
Люди всех групп одинаково часто берут кредиты, но чаще имели задолжности заемщики со средним уровнем дохода.
Люди с доходом выше среднего реже всего имели задолжности по кредитам, так как имеют достаточно стабильно высокий доход, и имеют накопления позволяющие в случае непредвиденных ситуаций выплачивать кредиты в срок.
Заемщики с низким уровнем дохода имеют меньше задолжностей чем у людей со средним, вероятнее всего из-за более низких сумм кредитования.</div>


<div>Как разные цели кредита влияют на его возврат в срок?</div>

In [26]:
data_purpose_groupby=data[['purpose_group','debt']]
data_purpose_groupby=data_purpose_groupby.groupby('purpose_group').sum()
data_purpose_groupby['quantity']=data['purpose_group'].value_counts()
try:
    data_purpose_groupby['percent']=(data_purpose_groupby['debt'] / data_purpose_groupby['quantity']).apply('{:.1%}'.format)
    print(data_purpose_groupby)
except:
    print('Проверьте правильность данных')


                        debt  quantity percent
purpose_group                                 
Кредит на автомобиль     400      4284    9.3%
Кредит на недвижимость   779     10764    7.2%
Кредит на образование    370      3995    9.3%
Кредит на свадьбу        184      2310    8.0%


<div>По сводной таблице цели кредита и наличия задолжностей наблюдаем, что чаще всего люди берут кредиты на недвижимость, и реже имели задолжности по этим кредитам. Это объясняется снижением ставок по кредитам на недвижимость, и большим объемом сделок по недвижимости в последнее время, так же влияет, что вложение в недвижимость считается одним из самых выгодных вложений средств. Низкий процент по задолжностям объясняется тем, что в случае появления таковых банк может забрать недвижимость у заемщика, либо будут начисляться большие пени.
Наиболее часто люди имели задолжности по кредитам на автомобили и образование.Вероятнее всего люди часто преобретают автомобили 'не по карману', стоимость и обслуживание которых не могут себе позволить. Люди берущие кредит на образование часто - это молодые люди, не умеющие планировать свой бюджет, и не имеющие финансовой стабильности, что приводит к появлению задолжностей.
Цели кредитования имеют большой уровень зависимости на его возрат в срок, разница чуть более 2%.</div>

In [27]:
import numpy
percent_mean = []
percent_mean.append((data_purpose_groupby['debt'] / data_purpose_groupby['quantity']).mean())
percent_mean.append((data_family_groupby['debt'] / data_family_groupby['quantity']).mean())
percent_mean.append((data_total_income_groupby['debt'] / data_total_income_groupby['quantity']).mean())
percent_mean.append((data_children_groupby['debt'] / data_children_groupby['quantity']).mean())
print('Средний процент, имеющих задолжности по возратам крeдитов '"{:.1%}".format(numpy.mean(percent_mean)))

Средний процент, имеющих задолжности по возратам крeдитов 8.2%


<div>Общий вывод</div>

<div>Целью данного иследования было выяление влияния семейного положения и количества детей клиента на факт погашения кредита в срок.

Входные данные от банка — статистика о платёжеспособности клиентов. 

Проверив данные на наличие пропусков обнаружили, что общий трудовой стаж и ежемесячный доход пропущен в 2174 случаях. Так же обнаружили черезмерно большие значения трудового стажа и пришли к выводу, что не все значения трудового стажа в формате дней. Еще обнаружили наличие отрицательных значений рабочего стажа. После чего привели эти значения к единообразному виду. 
Далее заполнили пропуски в столбцах ежемесечного дохода и трудового стажа, предварительно разбив на категории по возрасту и типу занятости. Пропуски заполняли средними значениями соответствующих категорий. 
В значениях возраста клиента обнаружили нулевые значения, и определив, что таких значений мало, удалили их. 
После обработки пропусков обратили внимание, что значения общего трудового стажа представлены вещественные числами, что не является корректным, заменили их на целочисленные значения. Так же значения ежемесячного дохода не нуждаются в такой высокой точности, поэтому их тоже заменили целочисленными значениями. 
Следующим шагом проверили данные на дублирования, обнаружили неочевидные дублирования в столбце образования. Все дублирующие строки убрали.
Можно предположить, что пропуски, дублирование данных, нулевые значения и другие артефакты возникли в результате ошибок при загрузке/выгрузке данных или при заполнении данных в таблицу, в результате человеческого фактора. Различный формат данных (дни и часы) в столбце трудового стажа мог произойти при сведении нескольких таблиц при заполнении которых использовались разные форматы.

Наше исследование показало, что средний процент, имеющих задолжности по возратам крeдитов 8.2%. Основной категорией кредитования является недвижимость, 50 % от всех кредитов, 58 % кредитов берут люди состоящие в браке, 66% всех кредитов берут люди не имеющие детей. При исследовании мы разбивали данные на 4 возрастные группы, на 7 групп ежемесячного дохода, на 3 группы по количеству детей и 3 категории занятости.

По результатам исследовая выяснили, что люди с 1-2 детьми чаще имели задолжности перед банками, а люди без детей реже имели задолжности. Люди не имеющие детей чаще всего обращаются за кредитами, что можно объяснить тем, что люди без детей меньше предрасположены к незапланированным тратам и могут быть более уверенными в своей платежеспособности, поэтому и вероятность задолжностей самая низкая. Задолжности группы с обычным количеством детей больше на 1.1 % чем у тех у кого более 2х детей, возможно из-за неопытности молодых родителей в планировании своих расходов, так и из-за того, что люди с большим количеством детей берут кредит только когда уверены, что смогут его вовремя погасить,что подтверждается и самым низким количеством взятия кретитов. Зависимость между наличием детей и возвратом кредита в срок значительна.

Так же, больше остальных берут кредиты люди находящиеся в браке, вероятнее всего из - за более стабильного бюджета,т.к в случае изменения доходов одного из семьи, сохраняется доход у второго, да и жизнь в браке предпологает постоянное улучшение качества жизни, с сопутсвующими затратами. Наивысшим процентом по наличию задолжностей владеют неженатые люди и люди живущие в гражданском браке. В первом случае, возможно, люди не находящиеся в браке берут кредит и попадая в финансовую нестабильность не имют финансовой подержки, из-за чего появляются задолжности. Часто неженатые люди - это молодые люди, не умеющие планировать свой бюджет, что тоже приводит к задолжностям. Люди живущие в гражданском браке чаще расходятся, чем люди женатые, тем самым по кредитам взятым 'на двоих' часто возникают задолжности. Зависимость между наличием задолжностей и семейным положением потенциальных заемщиков довольно велика, разница более 3%.

По ежемесячному доходу заемщиков выяснили, что люди всех групп одинаково часто берут кредиты, но чаще имели задолжности заемщики со средним уровнем дохода.
Люди с доходом выше среднего реже всего имели задолжности по кредитам, так как имеют достаточно стабильно высокий доход, и имеют накопления позволяющие в случае непредвиденных ситуаций выплачивать кредиты в срок.
Заемщики с низким уровнем дохода имеют меньше задолжностей чем у людей со средним, вероятнее всего из-за более низких сумм кредитования. Зависимость между уровнем дохода и возвратом кредита в срок значительна. Люди со средними доходами имеют разницу всего в 0.5%, а люди с самыми высоким и самыми низкими доходами отличаются от них на 1%, что достаточно много.

Далее определили, что чаще всего люди брали кредиты на недвижимость, и реже имели задолжности по этим кредитам. Это объясняется снижением ставок по кредитам на недвижимость, и большим объемом сделок по недвижимости в последнее время, так же влияет, что вложение в недвижимость считается одним из самых выгодных вложений средств. Низкий процент по задолжностям объясняется тем, что в случае появления таковых, банк может забрать недвижимость у заемщика, либо будут начисляться большие пени.
Наиболее часто люди имели задолжности по кредитам на автомобили и образование.Вероятнее всего люди часто приобретают автомобили 'не по карману', стоимость и обслуживание которых не могут себе позволить. Люди берущие кредит на образование часто - это молодые люди, не умеющие планировать свой бюджет, и не имеющие финансовой стабильности, что приводит к появлению задолжностей.
Цели кредитования имеют большой уровень зависимости на его возрат в срок, разница чуть более 2%.

Основной целью исследования было желание банка узнать, как различные критерии заемщиков (количество детей в семье, трудовой стаж, ежемесячный доход и др) влияют на вероятность появления задолженностей по возврату кредитов. Наши выводы позволят принимать решения по одобрению или отказу потенциальному заемщику в кредите.</div>